### Load dependencies 
We start by importing all necessary python packages, and the functionalities implemented in the utils folder of this repo.

In [ ]:
from utils.HN import *
from utils.model_eval import *
from utils.data_params import *

import random

### Insert dataset name, output directories, etc. (**TODO**)
Here we specify the dataset we want to test, and set the some paths. For the dataset, please use the abbreviations given in the paper. E.g., `data_set = BK-L23`. This will load the corresponding DoR-threshold, radii, etc. in the next cell (values can also be found in the paper). Please set the following variables:
- `data_set`: Name (abbreviation of the dataset).
- `imgs_dir`: Path to the directory containing the test images of the dataset.
- `model_weights`: Path to the trained model file (.pth).
- `output_dir`: Path to the directory ourtputs can be stored in (must exist, is NOT going to be created).

In [ ]:
data_set = ""
imgs_dir = "" 
model_weights = ""
output_dir = ""    

For this next cell, no input is required - just run it. 

In [ ]:

classID2name =  parameters[data_set]["classID2name_HN"]
dor_thresh = parameters[data_set]["dor_thresh"]
radii = parameters[data_set]["radii"]

### Run inference (no input required)
This cell will run HerdNet on the images from the “imgs_dir” entered above and will automatically create a folder inside it (HerdNet_results) in which the .csv file containing the detections will be saved.
- `dets_file`: This is the path to the .csv file containing the HerdNet predictions. One row represents one detection and is expected to have the following columns:
    - `images`: Contains the file names of the images.
    - `x`: Contains the x-coordinate of the detection.
    - `y`: Contains the y-coordinate of the detection.
    - `scores`: Contains the confidence score of the prediction.
    - `species`: Contains the name of the detected species.

In [ ]:
random.seed(0)

#%run -m utils.inference_herdnet {imgs_dir} {model_weights} --size 640 --over 128

from pathlib import Path

dets_file = next(Path(imgs_dir).rglob("*_HerdNet_detections.csv"), None)

### Compute Evaluation metrics (no input required)
This cell evaluated the ouputs of the previous cell. It will generate a number of files:
- `count_diffs_img_lvl.xlsx`: Excel sheet containing the difference between predicted and ground truth count for each image.
- `counts_gt_pred_*.png`: Plot of predicted vs. forund truth count for class `*`.
- `counts_total.json`: Predicted counts summed over all images.
- `em.json`: Evaluation metrics.
- `errors_img_lvl.json`: Counting metrics.
- `F1_curve.png`: F1 score plotted against the confidence threshold.
- `P_curve.png`: Precision plotted against the confidence threshold.
- `R_curve.png`: Recall plotted against the confidence threshold.

In [ ]:
class_name2id =  {name: cid for cid,name in classID2name.items()}

read_detections_HN(dets_file=dets_file, cls_name2id=class_name2id, imgs_dir=imgs_dir, dor_thresh=dor_thresh, radii=radii, class_ids=list(radii.keys()), output_dir=output_dir,
                   ann_file=f"{imgs_dir}/test_annotations.json", ann_format=parameters[data_set]["ann_format"])
compute_errors_img_lvl(gt_counts_dir=f"{imgs_dir}/image_counts", pred_counts_dir=f"{output_dir}/detections", class_ids=list(classID2name.keys()), 
                           output_dir=output_dir)
compute_em_img_lvl(preds_dir=f"{output_dir}/detections", class_id2name=classID2name, task="locate", output_dir=output_dir)    